# 🚀 Build an LLM Chatbot with LangChain + Chroma (Google Colab)
This Colab notebook demonstrates how to build a simple chatbot using open-source models and Chroma for retrieval.

In [ ]:
# --- 1. Install Dependencies ---
!pip install langchain chromadb sentence-transformers transformers accelerate bitsandbytes gradio

In [ ]:
# --- 2. Imports ---
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
# --- 3. Sample Document ---
sample_text = """
Welcome to the House Expense Manager App.
This application allows multiple members in a household to add and track expenses.
Each expense has a title, category, payer, participants, and optional receipt image.
It can also calculate splits automatically and generate spending statistics.
"""

with open("sample.txt", "w") as f:
    f.write(sample_text)

loader = TextLoader("sample.txt", encoding="utf-8")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"✅ Loaded {len(chunks)} chunks.")

In [ ]:
# --- 4. Create Embeddings & Vector Store ---
embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = Chroma.from_documents(chunks, embedding=embeddings)
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
# --- 5. Load Free/Open LLM ---
model_id = "HuggingFaceH4/zephyr-7b-alpha"  # Alternative: 'mistralai/Mistral-7B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto",
    load_in_8bit=True
)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
llm = HuggingFacePipeline(pipeline=pipe)

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [ ]:
# --- 6. Query Example ---
query = "How does the app split expenses?"
print("❓ Query:", query)
print("💬 Answer:", qa_chain.run(query))

In [ ]:
# --- 7. Optional Gradio Chat Interface ---
import gradio as gr

def chat_fn(query):
    return qa_chain.run(query)

iface = gr.Interface(fn=chat_fn, inputs="text", outputs="text", title="LangChain + Chroma Chatbot")
iface.launch()